In [ ]:
# Prompt: THIS MODEL IS ALREADY SAVED ON HUGGING FACE PLATFORM.
# I AM MAKING FINANCIAL CHATBOAT IN WHICH WHEN USER UPLOADS ANY PDF FOR THE QUESTIONS. 
# THERE IS BERT MODEL WHICH GIVES OUTPUT THAT WHETHER THE QUESTION IS SPAN OR ARITHMETIC. 
# NOW I WANT TO INTERPRETATE THE BERT MODEL ON TEXT DATA ONLY IN WHICH I WANT CODE THAT INTERPRETATE THE MODEL USING LIME ALGORTIHTM WHICH GIVE INFORMATION OF OUTPUT THAT MODEL PERFORMS WELL AGAINST OUTPUT OR NOT

In [ ]:


import lime
import lime.lime_text
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np

# Load your model and tokenizer
model_name = "rahul14/span-arithmetic-classification"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

# Prediction function for LIME
def predictor(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=32)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.nn.functional.softmax(outputs.logits, dim=1).cpu().numpy()

# Create LIME explainer
explainer = lime.lime_text.LimeTextExplainer(class_names=["arithmetic", "span"])

def interpret_question(question):
    def highlight_text(words, explanation):
        highlighted = []
        for word in words:
            weight = next((w for feature, w in explanation if feature.lower() in word.lower()), 0)
            if weight > 0.1:
                highlighted.append(f"[+{weight:.2f}]{word}")
            elif weight < -0.1:
                highlighted.append(f"[{weight:.2f}]{word}")
            else:
                highlighted.append(word)
        return highlighted

    try:
        # Prediction
        pred_probs = predictor([question])[0]
        predicted_class = np.argmax(pred_probs)
        confidence = pred_probs[predicted_class]
        
        print(f"\nQuestion: '{question}'")
        print(f"Predicted: {'arithmetic' if predicted_class == 0 else 'span'} (confidence: {confidence:.2%})")
        
        # Explanation
        exp = explainer.explain_instance(question, predictor, num_features=10, num_samples=500, top_labels=1)
        explanation = get_explanation(exp, predicted_class)
        
        print("\nTop features influencing the prediction:")
        for feature, weight in explanation:
            print(f"{feature:20} {weight:+.4f}")
        
        print("\nText highlighting:")
        print(' '.join(highlight_text(question.split(), explanation)))
        
        return exp
    
    except Exception as e:
        print(f"Error interpreting question: {str(e)}")
        return None

def get_explanation(exp, predicted_class):
    try:
        return exp.as_list(label=predicted_class)
    except KeyError:
        return exp.as_list(label=0) if len(exp.available_labels()) > 0 else []


# Example usage
sample_questions = [
    # "What is the difference between 2019 average rate of inflation and 2019 average rate of increase in salaries?",
    "How much was the closing net book amount for software under development in 2019?",
    "What was the percentage change in cost of software under development between 2018 and 2019?"
]

for question in sample_questions:
    _ = interpret_question(question)
    print("\n" + "="*80 + "\n")


Question: 'What is the difference between 2019 average rate of inflation and 2019 average rate of increase in salaries?'
Predicted: arithmetic (confidence: 99.93%)

Top features influencing the prediction:
average              +0.1809
difference           +0.1228
2019                 +0.0577
between              +0.0473
and                  +0.0448
rate                 +0.0389
of                   +0.0323
is                   -0.0258
increase             -0.0242
in                   +0.0115

Text highlighting:
What is the [+0.12]difference between 2019 [+0.18]average rate of inflation and 2019 [+0.18]average rate of increase in salaries?



Question: 'How much was the closing net book amount for software under development in 2019?'
Predicted: span (confidence: 98.56%)

Top features influencing the prediction:
was                  +0.0795
much                 -0.0339
How                  -0.0293
the                  +0.0221
for                  -0.0148
software             -0.0138
unde

In [ ]:
# File ~\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\lime\explanation.py:141, in Explanation.as_list(self, label, **kwargs)
    # 128 """Returns the explanation as a list.
    # 129 
    # 130 Args:
#    (...)    138     given by domain_mapper. Weight is a float.
#     139 """
#     140 label_to_use = label if self.mode == "classification" else self.dummy_label
# --> 141 ans = self.domain_mapper.map_exp_ids(self.local_exp[label_to_use], **kwargs)
#     142 ans = [(x[0], float(x[1])) for x in ans]
#     143 return ans

# THIS IS MY CODE YOU PROVIDES AND THE KEY ERROR GOT FROM THR CODE